---
sidebar_position: 2
---

# Quickstart

<!-- THE MARKDOWN (.md) FILE IS GENERATED FROM THE NOTEBOOK (.ipynb) FILE -->

In this quickstart guide, you will install Substratus into a Google Cloud Platform project. Then you'll explore how Substratus can be used to build and deploy Open Source LLMs.

NOTE: Support for AWS ([GitHub Issue #12](https://github.com/substratusai/substratus/issues/12)) and Azure ([GitHub Issue #63](https://github.com/substratusai/substratus/issues/63)) is planned. Give those issues a thumbs up if you would like to see them prioritized.

<!-- TODO: quickstart video -->

<div class="video-container">
  <iframe class="video" src="https://www.youtube.com/embed/dQw4w9WgXcQ" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>
</div>

## Prerequisites

You will need a [Google Cloud Platform](https://console.cloud.google.com/) project with billing enabled.

Run the commands below to make sure you have the required tools.

In [ ]:
!docker version || open 'https://docs.docker.com/get-docker/'

In [ ]:
!gcloud version || open 'https://cloud.google.com/sdk/docs/install'

In [ ]:
!gke-gcloud-auth-plugin --version || gcloud components install gke-gcloud-auth-plugin

In [ ]:
!kubectl version --client || open 'https://kubernetes.io/docs/tasks/tools/#kubectl'

## Install Substratus

Create a substratus GKE cluster along with supporting infrastructure (buckets, service accounts, image registries).

In [ ]:
!docker run -it \
  -v ${HOME}/.kube:/root/.kube \
  -e PROJECT=$(gcloud config get project) \
  -e TOKEN=$(gcloud auth print-access-token) \
  substratusai/installer gcp-up.sh

`kubectl` should now be pointing at the substratus cluster.

## Deploy an Open Source Model

To keep this quick, we'll use a smallish model, the falcon-7b-instruct model (just 7 billion parameters).

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/substratusai/substratus/main/examples/falcon-7b-instruct/base-model.yaml

The model is now being downloaded from HuggingFace into a GCS bucket. This takes about 5 minutes. 
Let's also deploy the built model by applying a Server manifest. Server should start serving shortly after the Model build finishes (~3 minutes).

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/substratusai/substratus/main/examples/falcon-7b-instruct/server.yaml

You can check on the progress of both processes using a single command.

In [ ]:
! kubectl get ai

When the Server reports a `Ready` status, proceed to the next section to test it out.

## Testing out the Server

The way every company chooses to expose a model will be different. In most cases models are integrated into other business applications and are rarely exposed directly to the Internet. By default, substratus will only serve the model within the Kubernetes cluster (with a Kubernetes [Service](https://kubernetes.io/docs/concepts/services-networking/service/) object). From here, it's up to you to expose the model to a wider network (e.g., the internal VPC network or the Internet) via annotated Service or Ingress objects.

In order to access the model for exploratory purposes, forward ports from within the cluster to your local machine.

In [4]:
!kubectl port-forward service/falcon-7b-instruct-server 8080:8080

Error from server (NotFound): services "falcon-7b-instruct-server" not found


All substratus Servers ship with an API and interactive frontend. Open up your browser to [http://localhost:8080/](http://localhost:8080/) and talk to your model! Alternatively, request text generation via the OpenAI compatible HTTP API:

In [ ]:
! curl http://localhost:8080/v1/completions \
  -H "Content-Type: application/json" \
  -d '{ \
    "model": "falcon-7b-instruct", \
    "prompt": "Who was the first president of the United States? ", \
    "max_tokens": 10\
  }'

If you are interested in continuing your journey through Substratus, take a look at the [Guided Walkthrough](./category/walkthrough) or
follow the [tutorial to finetune falcon-7b-instruct](./tutorials/deploying-finetuning-falcon-7b-instruct.md) with a custom dataset.

## Cleanup

The process that is serving the model can be stopped by simply deleting the same Server object that was applied before.

In [ ]:
!kubectl delete -f https://raw.githubusercontent.com/substratusai/substratus/main/examples/facebook-opt-125m/server.yaml

If you want to uninstall the entire Substratus system and all infrastructure, you can run the `gcp-down.sh` script from the installation container.

In [ ]:
!docker run -it \
  -e PROJECT=$(gcloud config get project) \
  -e TOKEN=$(gcloud auth print-access-token) \
  substratusai/installer gcp-down.sh

To learn more about how Substratus works, check out the [Architecture](./architecture) page.